# IAR6 Exercise

$\textbf{Cruise}:$ The robot drives straight forward. 

$\textbf{Avoid}:$ If an obstacle is detected on the left side, the robot turns right until the obstacle is no longer detected. If an obstacle is detected on the right side, the robot turns left. 

$\textbf{Escape}:$ If a collision is detected, the robot backs up for $t_b = 2~\textrm{s}$. Afterwards, it spins for $t_s = 2~\textrm{s}$.  


In [1]:
import time
from loop_rate_limiters import RateLimiter
import random

# import thymiodirect package
from thymiodirect import Connection 
from thymiodirect import Thymio

In [44]:
class Cruise:
    def __init__(self, robot):
        self.command = []
        self.robot = robot
        self.driving_factor = 100
        self.threshold = 50
    
    def action(self):
        print("Going Straight")
        self.forwards()
        return True

    def forwards(self):
        self.robot['motor.left.target'] = self.driving_factor
        self.robot['motor.right.target'] = self.driving_factor

class Escape:
    def __init__(self, robot):
        self.command = []
        self.robot = robot
        self.driving_factor = 100
        self.threshold = 200
        self.timer_1 = 0
        self.timer_2 = 0
        self.time_threshold_1 = 2
        self.time_threshold_2 = 4
        self.active = False
        self.direction = "left"

    def action(self):
        if max(robot['prox.ground.reflected'][:2]) > self.threshold:
            if not self.active:
                print("Starting escape")
                self.active = True
                self.timer_1 = time.time()+self.time_threshold_1
                self.timer_2 = time.time()+self.time_threshold_2
                self.direction = random.choice(["left", "right"])
        if self.timer_1 > time.time():
            print("Going Backwards")
            self.backwards()
            return True
        elif self.timer_2 > time.time():
            print(f"Turning {self.direction}")
            self.turn()
            return True
        else:
            self.active = False
            return False

    def turn(self):
        if self.direction == "left":
            self.robot['motor.left.target'] = self.driving_factor
            self.robot['motor.right.target'] = -self.driving_factor
        else:
            self.robot['motor.left.target'] = -self.driving_factor
            self.robot['motor.right.target'] = self.driving_factor
    def backwards(self):
        self.robot['motor.left.target'] = -self.driving_factor
        self.robot['motor.right.target'] = -self.driving_factor

class Avoid:
    def __init__(self, robot):
        self.robot = robot
        self.command = []
        self.driving_factor = 100
        self.threshold = 200
        self.direction = "left"
        
    def action(self):
        if max(robot['prox.horizontal'][:2]) > self.threshold:
            print("Detected obstacle left")
            self.direction = "left"
            self.turn()
        elif max(robot['prox.horizontal'][3:5]) > self.threshold:
            print("Detected obstacle right")
            self.direction = "right"
            self.turn()
        else:
            return False
        return True

    def turn(self):
        if self.direction == "left":
            self.robot['motor.left.target'] = self.driving_factor
            self.robot['motor.right.target'] = -self.driving_factor
        else:
            self.robot['motor.left.target'] = -self.driving_factor
            self.robot['motor.right.target'] = self.driving_factor

In [3]:
# Set to True to use the Thymio simulator
use_simulator = True

# Sets the rate limiter to control the loop frequency in Hz
rate = RateLimiter(frequency=10.0) 

if use_simulator:
    port = 2001
    th = Thymio(use_tcp=True, host='localhost', tcp_port=port, on_connect=lambda node_id: print(f' Thymio {node_id} is connected'))
else:
    port = Connection.serial_default_port()
    th = Thymio(serial_port=port, on_connect=lambda node_id: print(f' Thymio {node_id} is connected'))

# Connect to Robot
th.connect()
robot = th[th.first_node()]

# Delay to allow robot initialization of all variables
time.sleep(1)


 Thymio 1 is connected


In [45]:
# Arbiter
behaviors = []
        
start = True


behaviors.append(Escape(robot))
behaviors.append(Avoid(robot))
behaviors.append(Cruise(robot))



In [46]:
# don't delete the try-finally block, add your code inside the try block
time_counter = time.time()
try:
    # Main loop
    while True:
        if time_counter + 100 < time.time():
            start = False
        if robot["button.forward"] == 1:
            start = True
        if robot["button.center"] == 1:
            start = False
        print(f"{"-"*50}\nRobot at time {time.time()-time_counter} doing: ")
        if start:
            for behavior in behaviors:
                if behavior.action():
                    break
        else:
            robot['motor.left.target'] = 0
            robot['motor.right.target'] = 0
        rate.sleep()
finally:
    robot['motor.left.target'] = 0
    robot['motor.right.target'] = 0
    print("Interrupted")

[loop_rate_limiters] [13:44:50] [warning] rate limiter is late by 14162.5 [ms] (rate_limiter.py:96)


--------------------------------------------------
Robot at time 0.0002193450927734375 doing: 
Going Straight
--------------------------------------------------
Robot at time 0.001622915267944336 doing: 
Going Straight
--------------------------------------------------
Robot at time 0.10173797607421875 doing: 
Going Straight
--------------------------------------------------
Robot at time 0.20190167427062988 doing: 
Going Straight
--------------------------------------------------
Robot at time 0.3019754886627197 doing: 
Going Straight
--------------------------------------------------
Robot at time 0.4020383358001709 doing: 
Going Straight
--------------------------------------------------
Robot at time 0.502103328704834 doing: 
Going Straight
--------------------------------------------------
Robot at time 0.6021733283996582 doing: 
Going Straight
--------------------------------------------------
Robot at time 0.7024185657501221 doing: 
Going Straight
-------------------------------

KeyboardInterrupt: 